In [2]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

%matplotlib inline

In [3]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

def trend(time, slope=0):
    return slope * time

def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

In [4]:
time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, 0.1)  
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Create the series
series = (baseline 
          + trend(time, slope) 
          + seasonality(time, period=365, amplitude=amplitude))
# Update with noise
series += noise(time, noise_level, seed=42)

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [5]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(
        window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(
        lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)\
        .map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [6]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

In [7]:
dataset = windowed_dataset(
    x_train, window_size, batch_size, shuffle_buffer_size)

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                           input_shape=[None]),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 100.0)])

In [9]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, None, 1)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 64)          8704      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
_________________________________________________________________
lambda_1 (Lambda)            (None, 1)                 0         
Total params: 33,601
Trainable params: 33,601
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(dataset, epochs=100, callbacks=[lr_schedule])

Epoch 1/100
31/31 [==============================] - 9s 303ms/step - loss: 21.4949 - mae: 22.0115
Epoch 2/100
31/31 [==============================] - 3s 87ms/step - loss: 21.2168 - mae: 21.6440
Epoch 3/100
31/31 [==============================] - 3s 85ms/step - loss: 20.7111 - mae: 21.2278
Epoch 4/100
31/31 [==============================] - 3s 87ms/step - loss: 20.1562 - mae: 20.7739
Epoch 5/100
31/31 [==============================] - 3s 92ms/step - loss: 19.7903 - mae: 20.2843
Epoch 6/100
31/31 [==============================] - 3s 94ms/step - loss: 19.2615 - mae: 19.6776
Epoch 7/100
31/31 [==============================] - 3s 93ms/step - loss: 18.2681 - mae: 18.7308
Epoch 8/100
31/31 [==============================] - 3s 94ms/step - loss: 17.4057 - mae: 17.9757
Epoch 9/100
31/31 [==============================] - 3s 98ms/step - loss: 17.2263 - mae: 17.6581
Epoch 10/100
31/31 [==============================] - 3s 95ms/step - loss: 16.8978 - mae: 17.3457
Epoch 11/100
31/31 [========

31/31 [==============================] - 3s 96ms/step - loss: 7.5195 - mae: 8.0240
Epoch 86/100
31/31 [==============================] - 3s 97ms/step - loss: 6.5818 - mae: 7.1563
Epoch 87/100
31/31 [==============================] - 3s 96ms/step - loss: 12.1107 - mae: 12.5566
Epoch 88/100
31/31 [==============================] - 3s 94ms/step - loss: 9.0437 - mae: 9.4594
Epoch 89/100
31/31 [==============================] - 3s 96ms/step - loss: 6.9742 - mae: 7.5067
Epoch 90/100
31/31 [==============================] - 3s 96ms/step - loss: 6.7227 - mae: 7.1806
Epoch 91/100
31/31 [==============================] - 3s 96ms/step - loss: 8.2915 - mae: 8.7262
Epoch 92/100
31/31 [==============================] - 3s 95ms/step - loss: 6.0605 - mae: 6.5606
Epoch 93/100
31/31 [==============================] - 3s 96ms/step - loss: 8.7262 - mae: 9.4483
Epoch 94/100
31/31 [==============================] - 3s 96ms/step - loss: 12.0523 - mae: 12.6195
Epoch 95/100
31/31 [=============================

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 30])

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

In [ ]:
dataset = windowed_dataset(
    x_train, window_size, batch_size, shuffle_buffer_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                           input_shape=[None]),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 100.0)])
model.compile(loss="mae", 
              optimizer=tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9),
              metrics=["mae"])
model.summary()

In [ ]:
history = model.fit(dataset, epochs=500, verbose=0)

In [ ]:
forecast = []
results = []
for time in range(len(series) - window_size):
    forecast.append(
        model.predict(series[time:time + window_size][np.newaxis]))
forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]

plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid)
plot_series(time_valid, results)

In [ ]:
tf.keras.metrics.mean_absolute_error(x_valid, results).numpy()

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
mae=history.history['mae']
loss=history.history['loss']
epochs=range(len(loss)) # Get number of epochs

#------------------------------------------------
# Plot MAE and Loss
#------------------------------------------------
plt.plot(epochs, mae, 'r')
plt.plot(epochs, loss, 'b')
plt.title('MAE and Loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])

plt.figure()
epochs_zoom = epochs[200:]
mae_zoom = mae[200:]
loss_zoom = loss[200:]

#------------------------------------------------
# Plot Zoomed MAE and Loss
#------------------------------------------------
plt.plot(epochs_zoom, mae_zoom, 'r')
plt.plot(epochs_zoom, loss_zoom, 'b')
plt.title('MAE and Loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])

In [ ]:
tf.keras.backend.clear_session()
dataset = windowed_dataset(
    x_train, window_size, batch_size, shuffle_buffer_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                           input_shape=[None]),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 100.0)])

model.compile(loss="mae", 
              optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9))
model.fit(dataset, epochs=100, verbose=0)

In [ ]:
tf.keras.backend.clear_session()
dataset = windowed_dataset(
    x_train, window_size, batch_size, shuffle_buffer_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                           input_shape=[None]),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 100.0)])

model.compile(loss="mae", 
              optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9))
model.fit(dataset,epochs=100)